# Réalisez une application mobile de recommandation de contenu
## Partie II) Content based

## INTRODUCTION
Dans cette second partie nous prenons une autre approche basé directment sur les articles avec un algorithme de similarité de cosinus. 

## SOMMAIRE

### I) Chargement des données
### II) Modélisations
### III) Calcul de similarité
### IV) Recommandation d'articles

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### I) Chargement des données
on configure les données nécéssaire et on définie les variables et importations.

In [ ]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import pickle
import math
from time import localtime, strftime, time
from datetime import timedelta

from sklearn.metrics.pairwise import cosine_similarity

# import tensorflow as tf

In [ ]:
LOCAL_COMPUTING = False

In [ ]:
if LOCAL_COMPUTING:
    ROOT_PATH = ''
else:
    ROOT_PATH = '/content/drive/MyDrive/Colab Notebooks/P9'

In [ ]:
ARTICLE_TO_RETRIEVE_NB = 5
ARTICLE_BASIS_LOW = 220000
ARTICLE_BASIS_HIGH = 270000
REMOVE_READ_ARTICLES = True

Etant données que cette algorithme est très lourd en exécution, on réduit notre jeu de donnée.

In [ ]:
art_emb = pickle.load(open(os.path.join(ROOT_PATH,'donnes_projet/articles_embeddings.pickle'), 'rb'))
art_emb

array([[-0.16118301, -0.95723313, -0.13794445, ..., -0.231686  ,
         0.5974159 ,  0.40962312],
       [-0.52321565, -0.974058  ,  0.73860806, ...,  0.18282819,
         0.39708954, -0.83436364],
       [-0.61961854, -0.9729604 , -0.20736018, ..., -0.44758022,
         0.8059317 , -0.28528407],
       ...,
       [-0.25139043, -0.9762427 ,  0.58609664, ..., -0.14372464,
         0.06809307, -0.7050104 ],
       [ 0.22434181, -0.92328775, -0.38174152, ...,  0.6871319 ,
        -0.5315117 ,  0.01072566],
       [-0.25713393, -0.9946313 ,  0.9837918 , ...,  0.98387307,
        -0.8381829 , -0.1792827 ]], dtype=float32)

In [ ]:
ref_df = pd.read_csv(os.path.join(ROOT_PATH,'data.csv'), sep=',', encoding='utf-8', dtype='int64')
ref_df

article_id  category_id  ...  user_id  click_timestamp
0                 3            1  ...   246630    1507572378399
1                27            1  ...    58062    1507144866293
2                69            1  ...   289974    1507832069718
3                81            1  ...    65902    1506954288811
4                81            1  ...    56070    1506974215111
...             ...          ...  ...      ...              ...
2988176      364043          460  ...   277901    1507735330786
2988177      364043          460  ...   317423    1508163146474
2988178      364043          460  ...    93792    1508181241509
2988179      364046          460  ...   126974    1507037561633
2988180      364046          460  ...   303872    1507977283025

[2988181 rows x 8 columns]

In [ ]:
art_mdata = pd.read_csv(os.path.join(ROOT_PATH,'donnes_projet/articles_metadata.csv'), sep=',', encoding='utf-8')
art_mdata

article_id  category_id  created_at_ts  publisher_id  words_count
0                0            0  1513144419000             0          168
1                1            1  1405341936000             0          189
2                2            1  1408667706000             0          250
3                3            1  1408468313000             0          230
4                4            1  1407071171000             0          162
...            ...          ...            ...           ...          ...
364042      364042          460  1434034118000             0          144
364043      364043          460  1434148472000             0          463
364044      364044          460  1457974279000             0          177
364045      364045          460  1515964737000             0          126
364046      364046          460  1505811330000             0          479

[364047 rows x 5 columns]

On préparer nos dataframes, celui des articles et ce des utilisateurs avec les références des articles. L'essentiel ici est d'avoir acces aux articles des utilisateurs pourvoir leur proposé des articles similaire à ce qu'il on déjà lu en fonction de leur similarité avec les autres articles.

### II) Modélisations

On commence par déterminer les articles favoris de l'utilisateur, ce qu'il a le plus lu recemment, ce sera le role de la fonction **get_best_user_articles**.  
La fonction **compute_article_similarities** se charge de calculer la similarité entre les articles (cosine similarity) et enfin on obtient les recommandations de l'utilisateur avec **get_user_recommendations**.

In [ ]:
def get_best_user_articles(user_id, art_low, art_high):
    user_articles = ref_df[['click_article_id','click_timestamp']][ref_df['user_id'] == user_id]
    user_articles = user_articles[(user_articles['click_article_id'] > art_low) & (user_articles['click_article_id'] < art_high)]
    user_articles_views = user_articles['click_article_id'].value_counts()
    user_articles_all = user_articles['click_article_id'].unique()
    #print('• user article view nb : \n{}'.format(user_articles_views[:5]))
    #print('• all user articles : \n{}'.format(user_articles_all))
    max_views = user_articles_views.max()
    #print('\n• max views : {}'.format(max_views))
    max_viewed_articles = user_articles_views[(user_articles_views == max_views) == True].index.to_list()
    #print('• max viewed user articles : {}'.format(max_viewed_articles))

    max_viewed_articles_df = user_articles[user_articles['click_article_id'].isin(max_viewed_articles)]
    sorted_max_viewed_articles_df = max_viewed_articles_df.groupby(['click_article_id']).max().sort_values(by='click_timestamp', ascending=False)
    #print(sorted_max_viewed_articles_df)
    #print(sorted_max_viewed_articles_df.index.to_list())

    return sorted_max_viewed_articles_df.index.to_list(), max_views, user_articles_all

In [ ]:
def get_user_recommendations(user_id, recom_nb, art_sim, art_low, art_high, remove_read_art = True):

    start_time = time()
    print('• user recommendation starts at : {}\n\n---'.format(strftime("%d %b %Y %H:%M:%S", localtime(start_time))))

    top_read_art, max_views, all_read_art = get_best_user_articles(user_id, art_low, art_high)
    print('• top read articles : {}'.format(top_read_art))
    print('• max views of top read articles = {}'.format(max_views))
    #print('• all read articles : {}'.format(all_read_art))

    user_recommandations = pd.DataFrame(columns=['art_recom','sim_score','art_ref', 'words_count'])

    i = 0
    while i < recom_nb:

        ref_article_in_list = i%len(top_read_art)
        # +1 to avoid ref article in list w/ highest score (= 1)
        
        if remove_read_art:
            top_to_get = math.floor(i/len(top_read_art))
        else:
            top_to_get = math.floor(i/len(top_read_art)) + 1

        #print(top_read_art[i%len(top_read_art)])
        #print('ref_article_in_list :', ref_article_in_list)
        #print('top_to_get :', top_to_get)

        ref_article_sim = top_read_art[ref_article_in_list] - art_low
        #print(('ref_article_sim : {}'.format(ref_article_sim)))
        

        sim_scores = list(enumerate(art_sim[ref_article_sim]))
        #print('sim_scores :', len(sim_scores))

        if remove_read_art:
            sim_scores = [x for x in sim_scores if (x[0] + art_low) not in all_read_art]
            #print('sim_scores :', len(sim_scores))

        sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
        #print('sim_scores :', sim_scores[:10])

        recom_article = sim_scores[top_to_get]
        #recom_article = (recom_article[0] + art_low, recom_article[1], top_read_art[ref_article_in_list])
        #print('recom_article : {}'.format(recom_article))
        #print('\n')
        # ref_df[ref_df.article_id.isin[art]
        user_recommandations.loc[user_recommandations.shape[0]] = [
            recom_article[0] + art_low,
            recom_article[1],
            top_read_art[ref_article_in_list],
            ref_df[ref_df.click_article_id ==([25181][0] + 220000)]["words_count"].head(1).values[0],
        ]
        i += 1

    end_time = time()
    print('---\n\n• user recommendation ends at : {}'.format(strftime("%d %b %Y %H:%M:%S", localtime(end_time))))
    print('•• user recommendation lasts : {}'.format(str(timedelta(seconds=(end_time-start_time)))))

    user_recommandations['art_recom'] = user_recommandations['art_recom'].astype('int32')
    user_recommandations['art_ref'] = user_recommandations['art_ref'].astype('int32')
    user_recommandations['words_count'] = user_recommandations['words_count'].astype('int32')

    return user_recommandations

In [ ]:
def enrich_recommendations(user_recom, cat_map):
    user_recom['cat_recom'] = 0
    user_recom['cat_ref'] = 0

    for i in tqdm(range(user_recom.shape[0])):
        user_recom['cat_recom'].iloc[i] = cat_map['category_id'][cat_map['article_id'] == user_recom.iloc[i]['art_recom']].min()
        user_recom['cat_ref'].iloc[i] = cat_map['category_id'][cat_map['article_id'] == user_recom.iloc[i]['art_ref']].min()

    user_recom['cat_recom'] = user_recom['cat_recom'].fillna(0)
    user_recom['cat_recom'] = user_recom['cat_recom'].astype('int16')

    return user_recom

In [ ]:
def compute_article_similarities(art_emb, art_low, art_high):

    start_time = time()
    print('• Cosine Similarity computing starts at : {}\n\n---'.format(strftime("%d %b %Y %H:%M:%S", localtime(start_time))))

    article_similarities = cosine_similarity(art_emb[art_low:art_high], art_emb[art_low:art_high])
    print('• article_similarities shape : {}'.format(article_similarities.shape))

    end_time = time()
    print('---\n\n• Cosine Similarity computing ends at : {}'.format(strftime("%d %b %Y %H:%M:%S", localtime(end_time))))
    print('•• Cosine Similarity computing lasts : {}'.format(str(timedelta(seconds=(end_time-start_time)))))

    return article_similarities

### III) Calcul de similarité

On calcul la similarité entre tous les articles de notre dataframe (la partie qu'on à prise)

In [ ]:
article_similarities = compute_article_similarities(art_emb, ARTICLE_BASIS_LOW, ARTICLE_BASIS_HIGH)
article_similarities

• Cosine Similarity computing starts at : 16 Aug 2021 16:59:41

---
• article_similarities shape : (50000, 50000)
---

• Cosine Similarity computing ends at : 16 Aug 2021 16:59:52
•• Cosine Similarity computing lasts : 0:00:10.753384


array([[0.99999976, 0.5165724 , 0.53477615, ..., 0.23342983, 0.18522803,
        0.26704332],
       [0.5165724 , 1.0000002 , 0.88190293, ..., 0.32728097, 0.22958705,
        0.27914646],
       [0.53477615, 0.88190293, 1.        , ..., 0.2906914 , 0.14946519,
        0.23436777],
       ...,
       [0.23342983, 0.32728097, 0.2906914 , ..., 1.0000002 , 0.7829189 ,
        0.83609027],
       [0.18522803, 0.22958705, 0.14946519, ..., 0.7829189 , 1.        ,
        0.74915063],
       [0.26704332, 0.27914646, 0.23436777, ..., 0.83609027, 0.74915063,
        0.9999999 ]], dtype=float32)

In [ ]:
article_similarities.shape

(50000, 50000)

### IV) Recommandation d'articles

Voyons les recommandations d'un utilisateur.

In [ ]:
USER_ID = 2300

In [ ]:
user_recommendations = get_user_recommendations(USER_ID, ARTICLE_TO_RETRIEVE_NB, article_similarities, ARTICLE_BASIS_LOW, ARTICLE_BASIS_HIGH, REMOVE_READ_ARTICLES)

• user recommendation starts at : 16 Aug 2021 17:00:10

---
• top read articles : [256565, 235840]
• max views of top read articles = 1
---

• user recommendation ends at : 16 Aug 2021 17:00:12
•• user recommendation lasts : 0:00:01.537181


In [ ]:
user_recommendations

art_recom  sim_score  art_ref  words_count
0     256727   0.918272   256565          150
1     235312   0.886773   235840          150
2     256866   0.914438   256565          150
3     235410   0.886702   235840          150
4     256414   0.911953   256565          150

On a les 5 articles recommendés pour l'utilisateur 2300.

In [ ]:
user_recommendations_w_cat = enrich_recommendations(user_recommendations, ref_df[['article_id','category_id']])
user_recommendations_w_cat

  0%|          | 0/5 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

art_recom  sim_score  art_ref  words_count  cat_recom  cat_ref
0     256727   0.918272   256565          150        390      390
1     235312   0.886773   235840          150        375      375
2     256866   0.914438   256565          150          0      390
3     235410   0.886702   235840          150          0      375
4     256414   0.911953   256565          150          0      390

Les articles recommendés sont très similiares au niveau des catégories et des nombre de mots qu'ils contiennent.

In [ ]:
user_recommendations_w_cat = enrich_recommendations(user_recommendations, art_mdata[['article_id','category_id']])
user_recommendations_w_cat

100%|██████████| 8/8 [00:00<00:00, 352.97it/s]


art_recom  sim_score  art_ref  cat_recom  cat_ref
0     118495   0.917994   119592        247      247
1     145453   0.863692   145166        269      269
2     118614   0.912347   119592        247      247
3     130280   0.855649   145166        253      269
4     118498   0.906862   119592        247      247
5     130463   0.850105   145166        253      269
6     118494   0.906186   119592        247      247
7     144081   0.848503   145166        269      269

### V) Evaluation du modèle

Contrairement à notre modèle de la partie précédente, cette évaluation se fait sur un petit échantillons. Cette évaluation n'est pas tout à fait correct mais on le fait dans le bus de voir rapidement les performance de ce modèle pour se faire une idée.

In [ ]:
# eval_set = 
# ref_df[['click_article_id','click_timestamp']][ref_df['user_id'] == user_id]

eval_set = ref_df[(ref_df['click_article_id'] > ARTICLE_BASIS_LOW) & (ref_df['click_article_id'] < ARTICLE_BASIS_HIGH)]
eval_set = eval_set.drop_duplicates(subset="user_id").user_id.sample(1000, random_state=16).values
eval_set

array([ 36328, 265291,  99870, 144799, 167578, 229911,  48916,  93075,
        66640,  79941, 155561, 107047, 180871,  11747, 120357,  25178,
        84003, 138427,  54446, 159926,  30168, 270438, 125574,  41160,
       112344,  14581,  39249, 305530, 148182,  42492,  27006,  37584,
       146730,  28045, 259622, 145158, 250848, 245404,  18970, 190009,
        87492,  75952, 183485, 107860, 151006,   2708,  34041, 310128,
       159089,  39184, 133029, 153129,  98667,   2219, 120971, 257043,
        90575, 272958, 113937,  31513,  69378, 199296,  46849,  81461,
        86205,  10309, 197425, 197690, 163837, 241829,   4483, 248436,
       238528,  78850,   7709,   8973, 152694,  70024, 121328,  24987,
       268506, 299762,  82391, 142147,  66322, 199431,  72559, 289778,
       119735, 142863,  63424,  53255, 140150, 173940,  13829, 109381,
       266478,  99333, 196665,   9819, 269463,  18036,  79614, 268356,
       158378,  33049, 128060, 274442, 176274,  35935,  75863,   8990,
      

On prend 1000 utilisateur au harsard dans les données, calcul leur recommandations. Nous n'avons pas de valeur réel avec quoi comparé les recommandations du modèle, on prend donc 1 comme valeur de référence qui la valeur maximale quand 2 articles sont similaires.

In [ ]:
# Evaluation modèle
y_pred = []
for users_ in tqdm(eval_set, total=len(eval_set)):
  user_recom = get_user_recommendations(users_, ARTICLE_TO_RETRIEVE_NB, article_similarities, ARTICLE_BASIS_LOW, ARTICLE_BASIS_HIGH, REMOVE_READ_ARTICLES)
  y_pred.append(user_recom.sim_score)

  0%|          | 0/1000 [00:00<?, ?it/s]

• user recommendation starts at : 16 Aug 2021 17:00:58

---
• top read articles : [234449]
• max views of top read articles = 1


  0%|          | 1/1000 [00:01<19:17,  1.16s/it]

---

• user recommendation ends at : 16 Aug 2021 17:00:59
•• user recommendation lasts : 0:00:01.142816
• user recommendation starts at : 16 Aug 2021 17:00:59

---
• top read articles : [235616, 234267, 235870, 234698]
• max views of top read articles = 1


  0%|          | 2/1000 [00:02<19:15,  1.16s/it]

...

---

• user recommendation ends at : 16 Aug 2021 17:20:17
•• user recommendation lasts : 0:00:01.122421
• user recommendation starts at : 16 Aug 2021 17:20:17

---
• top read articles : [234267]
• max views of top read articles = 1


100%|█████████▉| 999/1000 [19:20<00:01,  1.16s/it]

---

• user recommendation ends at : 16 Aug 2021 17:20:18
•• user recommendation lasts : 0:00:01.150945
• user recommendation starts at : 16 Aug 2021 17:20:18

---
• top read articles : [233717, 257291]
• max views of top read articles = 1


100%|██████████| 1000/1000 [19:21<00:00,  1.16s/it]

---

• user recommendation ends at : 16 Aug 2021 17:20:19
•• user recommendation lasts : 0:00:01.155882


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error 

In [ ]:
pred_val = []
for val in tqdm(y_pred, total=len(y_pred)):
  # for item in val.values.tolist():
  pred_val.append(val[0])
print(len(pred_val))
pred_val

100%|██████████| 1000/1000 [00:00<00:00, 55625.90it/s]

1000


[0.8736383318901062,
 0.9210278987884521,
 0.7790771722793579,
 0.8947902321815491,
 0.9203599095344543,
 0.9256427884101868,
 0.7790771722793579,
 0.8461546301841736,
 0.8461546301841736,
 0.8236939907073975,
 0.8325853943824768,
 0.7790771722793579,
 0.9221780300140381,
 0.972188413143158,
 0.8236939907073975,
 0.9022365212440491,
 0.8442029356956482,
 0.9588903188705444,
 0.7790771722793579,
 0.8461546301841736,
 0.9210278987884521,
 0.9106730222702026,
 0.8794234395027161,
 0.9388023018836975,
 0.9174015522003174,
 0.9225035905838013,
 0.8236939907073975,
 0.9155435562133789,
 0.8947902321815491,
 0.8861467242240906,
 0.9225035905838013,
 0.9209229350090027,
 0.750836968421936,
 0.9225035905838013,
 0.9010682702064514,
 0.7790771722793579,
 0.8751944303512573,
 0.9202626347541809,
 0.7790771722793579,
 0.9228637218475342,
 0.8442029356956482,
 0.9246727228164673,
 0.9210278987884521,
 0.9203149676322937,
 0.9345493316650391,
 0.9210278987884521,
 0.9071072936058044,
 0.850931942462

In [ ]:
len(pred_val)

1000

On calcul le score RMSE et MAE pour cette échantillons de 1000 utilisateur.

In [ ]:
y_true = np.ones(1000)
RMSE = mean_squared_error(y_true, pred_val)
MAE = mean_absolute_error(y_true, pred_val)
print("RMSE: ", RMSE, "\n MEA :", MAE)

RMSE:  0.016640216405269314 
 MEA : 0.11797400951385498


Voici les résultats des deux modèles mis bout-à-bout.

Scores content-based filtering
- RMSE : 0.016
- MAE : 0.12  

score collaborative filtering

- RMSE : 0.12 
- MAE : 0.03

## Conclusion:

On vient de développez une second approche de système de recommandation basé sur les articles (content-based) et leur similarités.   
Ce modèle demande beaucoup plus de temps pour l'entrainement raison pour la quelle on à pris une petite partie des données contrairement au premier modèle.  
Notre choix se porte sur le premier modèle car l'intégralité des données on pu être utilisées assez facilement et le modèle est plutôt rapide pour l'entrainement, même si les scores d'évaluation sont sensiblement meilleurs pour le modèle content-based.  
En somme le premier modèle nous offre plus de flexibilité par la rapidité de prédiction ce qui nous permet d'envisagé de calculer les recommandations en amont et de les stockés pour les ressortir assez rapidement mais également de faire des mis à jour assez souvent et tout cela assez rapidement (pratique pour les nouveaux utilisateurs et ou articles).